# Homework Week 1

https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Getting the data

Now let's get the FAQ data. You can run this snippet:

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [33]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

Answer: `index`

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [14]:
from elasticsearch import Elasticsearch

In [15]:
es_client = Elasticsearch("http://localhost:9200")

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-questions"

In [17]:
# Delete the index if it exists
# es_client.indices.delete(index=index_name)

# Now, create the index anew
es_client.indices.create(index=index_name, body=index_settings)
print("Index created.")


Index created.


In [18]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 45.22it/s]


In [34]:
def elastic_search_score(query, index_name):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = {
        'result': [],
        'score': []
    }
    
    for hit in response['hits']['hits']:
        result_docs['result'].append(hit['_source'])
        result_docs['score'].append(hit['_score'])

    return result_docs

In [35]:
# Example of calling the function
query = 'How do I execute a command in a running docker container?'
response = elastic_search_score(query, index_name)

top_score = response['score'][0]

In [36]:
print(f"Answer: Top record's score: {top_score}")

Answer: Top record's score: 84.050095


## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [133]:
def elastic_search_top3_ml(query, index_name="homework-questions"):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    }
                ],
                "filter": [
                    {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                ]
            }
        }
    }
    
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = {
        'result': [],
        'score': [],
        'question': []
    }

    for hit in response['hits']['hits'][:3]:
        result_docs['result'].append(hit['_source'])  # Storing the entire source document
        result_docs['score'].append(hit['_score'])    # Storing the score
        result_docs['question'].append(hit['_source'].get('question', 'No question available'))  # Safely accessing the question field

    return result_docs

In [134]:
query = 'How do I execute a command in a running docker container?'
q4_response = elastic_search_top3_ml(query)

top3_result = q4_response['question'][2]

In [135]:
print(f"Answer: {top3_result}")

Answer: How do I copy files from a different folder into docker container’s working directory?


In [136]:
q4_response['result']

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462


In [137]:
def build_prompt(query, search_results):
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""

    for doc in search_results['result']:
        # Safely get 'question' and 'text' from the document
        # question = doc.get('question', 'No question available')
        # text = doc.get('text', 'No text available')
        question = doc['question']
        text = doc['text']
        
        # Format the context string
        context = context + f"{context_template.format(question=question, text=text).strip()}\n\n"

    prompt =  prompt_template.format(question=query, context=context).strip()

    return prompt

In [138]:
prompt = build_prompt(query, q4_response)

In [139]:
print(f"Answer: Top record's score: {len(prompt)}")

Answer: Top record's score: 1462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```


In [105]:
import tiktoken

In [106]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [111]:
# Tokenize the text using the optimized tokenizer
tokens = encoding.encode(prompt)

322

In [125]:
print(f"Answer: How many tokens does our prompt have? -> {len(tokens)}")

Answer: How many tokens does our prompt have? -> 322


In [ ]:
decoded_tokens = encoding.decode(tokens)
decoded_tokens_safe = [encoding.decode_single_token_bytes(token) for token in tokens]

# print(decoded_tokens)

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [140]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [141]:
def rag(query):
    search_result = elastic_search_top3_ml(query, index_name="homework-questions")
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)

    return answer

In [143]:
from openai import OpenAI

In [144]:
client = OpenAI()

In [145]:
query = 'How do I execute a command in a running docker container?'

print(rag(query))

To execute a command in a running Docker container, you would use the `docker exec` command. First, you need to find the container ID of the running Docker container. Here are the steps you should follow:

1. List all running containers to find the container ID using the command:
   ```sh
   docker ps
   ```

2. Once you have the container ID, you can execute a command inside the running container. For example, to start a bash session, you would use:
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container you want to interact with.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: \$0.005 / 1K tokens
* Output: \$0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.